In [1]:
import astroquery.jplhorizons as jpl
import astropy.time as time
import astropy.units as u
import numpy as np
import pandas as pd
import urllib.request
import read_web
from datetime import datetime
import re

In [25]:
# Get vectors for spacecraft from JPL Horizons

def getVectors(object, start, stop, step):
    obj = jpl.Horizons(id=object, location='500@0', epochs={'start':start, 'stop':stop, 'step':step})
    return obj.vectors()

# Get vectors for object centered on current date
# plus/minus a specified number of days
# and for a specified time step
def getVectorsNow(object, start_diff, step):
    # get current date using astropy.time
    now = time.Time.now()
    # Get start/end dates
    if step.endswith('d'):
        start_diff = int(start_diff.split('d')[0])
        print(start_diff)
        start = now - start_diff*u.day
        stop = now + start_diff*u.day
    elif step.endswith('h'):
        start_diff = start_diff.split('h')[0]
        start = now - start_diff*u.hour
        stop = now + start_diff*u.hour
    elif step.endswith('m'):
        start_diff = start_diff.split('m')[0]
        start = now - start_diff*u.minute
        stop = now + start_diff*u.minute
    else:
        print('Error: time step must end with d, h, or m')
        return
    start = start.iso
    stop = stop.iso
    obj = jpl.Horizons(id=object, location='500@0', epochs={'start':start, 'stop':stop, 'step':step})
    return obj.vectors()

In [ ]:
def breakObjectList(spacecraft, minAU, maxAU):
    # If spacecraft is closer than minAU, add to freqList
    # If spacecraft is farther than maxAU, add to slowList
    # If spacecraft is between minAU and maxAU, add to midList
    freqList = []
    midList = []
    slowList = []

    for i in range(len(spacecraft)):
        if spacecraft['range'][i] < minAU:
            freqList.append(spacecraft[i])
        elif spacecraft['range'][i] > maxAU:
            slowList.append(spacecraft[i])
        else:
            midList.append(spacecraft[i])

    return freqList, midList, slowList

In [29]:
# Get current date using astropy.time
now = time.Time.now()
# Get only date from now, not time
now = now.iso.split(' ')[0]
now

'2023-09-23'

In [7]:
# Get single vector for a given object
def getSingleVector(object):
    # Get current date using astropy.time
    now = time.Time.now()
    # Get only date from now, not time
    now = now.iso.split(' ')[0]
    vector = jpl.Horizons(id=object, location='500@0', epochs={now})
    vector = vector.vectors()
    return vector

def getSingleRange(object):
    vector = getSingleVector(object)
    return vector['range'][0]

In [21]:
spacecraft = ['Voyager 1', 'Voyager 2', 'Mars Express', 'Juno', 'Cassini', "Explorer 1", "Cluster 4"]

In [22]:
for craft in spacecraft:
    try :
        print(craft, getSingleRange(craft), "AU")
    except ValueError as e:
        print("1 Fail")
        if 'Ambiguous target name' in str(e):
            print(e)
            try:
                print(craft, getSingleRange(craft+' (spacecraft)'), "AU")
            except ValueError as e:
                if "No ephemeris" and "after" in str(e):
                    print("No current ephemeris for", craft)
                else:
                    print("No ephemeris found for ", craft)
        else:
            print("No ephemeris found for", craft)

Voyager 1 161.1562165650144 AU
Voyager 2 134.496458509848 AU
Mars Express 1.611846516214913 AU
1 Fail
Ambiguous target name; provide unique id:
  ID#      Name                               Designation  IAU/aliases/other   
  -------  ---------------------------------- -----------  ------------------- 
      -61  Juno (spacecraft)                                                    
     -610  Juno Centaur Stage (spacecraft)                                      
 
Juno 4.955727320890237 AU
1 Fail
Ambiguous target name; provide unique id:
  ID#      Name                               Designation  IAU/aliases/other   
  -------  ---------------------------------- -----------  ------------------- 
      -82  Cassini (spacecraft)                                                 
     -150  Cassini Huygens (spacecraft)                                         
 
No current ephemeris for Cassini
1 Fail
No ephemeris found for Explorer 1
1 Fail
No ephemeris found for Cluster 4
